In [1]:

import re,string
import numpy as np
import pandas as pd 
# import fasttext
import spacy


import nltk
import networkx as nx
from nltk.corpus import wordnet,stopwords
from nltk.stem import WordNetLemmatizer 
from nltk import WordNetLemmatizer, pos_tag, word_tokenize

stop_words=stopwords.words('english')
lemmatizer = WordNetLemmatizer()

import gensim
from gensim.models import word2vec

from gensim.models.fasttext import FastText

import tensorflow as tf
from tensorflow.keras import backend

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm


from keras.layers import Activation, Embedding, Reshape,Concatenate
from keras.models import Sequential
from keras.preprocessing.sequence import skipgrams, make_sampling_table
from keras.preprocessing.text import Tokenizer

from typing import NamedTuple, List
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize


# import networkx as nx
from itertools import combinations
from collections import defaultdict  
import abc
from collections import Counter    
# nlp = spacy.load('en_core_web_sm')



C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [5]:
pat_data=pd.read_excel('../data/recent_5k.xlsx')
# pat_data=pd.read_excel('../data/Keyword_validation.xlsx')
pat_data

,glucose,college,country,grantID,Disease,center_copy_txt,Publication_Type_copy_txt,mesh_keyword_identified,organization_type_1st_level,mesh_term_qualified_desc,...,hospital_copy_txt,college_copy_txt,Journal_Title,Journal_Title_copy_txt,organization_name_1st_level_copy_txt,country_name_copy_txt,mesh_term_descriptor_name_copy_txt,category,PMID,tags_copy_txt
0,2021-08-06T01:24:45Z,NaN,NaN,NaN,NaN,NaN,journal article,"ciprofloxacin,cipro,staphylococcus,vancomycin,...",NaN,NaN,...,NaN,NaN,Foodborne Pathogens and Disease,Foodborne Pathogens and Disease,NaN,"republic of,republic of,republic of,republic o...",NaN,"Chemicals and Drugs,Organisms,Phenomena and Pr...",33769832.0,"2010-2018,isolates methicillin,chicken carcass..."
1,2021-08-06T01:24:46Z,NaN,NaN,NaN,diabetes,NaN,"journal article,research support\, non-u.s. gov't",NaN,"university,university,university,university","diabetes mellitus\, type 2,glucagon-like pepti...",...,NaN,NaN,Journal of Medicinal Chemistry,Journal of Medicinal Chemistry,"sungkyunkwan university,sungkyunkwan universit...",NaN,"animals,diabetes mellitus\, type 2,glucagon-li...","Organisms,Diseases,Chemicals and Drugs,Phenome...",33769827.0,"free fatty acid,pancreatic cell,glucagon,fatty..."
2,2021-08-06T01:24:47Z,south carolina college of pharmacy,NaN,NaN,"memory dysfunction,neurodegenerative diseases,...",NaN,"journal article,research support\, non-u.s. gov't",NaN,"university,university,university,university,un...","alzheimer disease,coumaric acids,histone deace...",...,NaN,south carolina college of pharmacy,Journal of Medicinal Chemistry,Journal of Medicinal Chemistry,"julius maximilian university of würzburg,medic...","germany,united states,germany,italy,germany,ge...","alzheimer disease,animals,catalytic domain,cel...","Diseases,Psychiatry and Psychology,Organisms,P...",33769811.0,"melatonin ferulic acid,dpph radical scavenging..."
3,2021-08-06T01:24:51Z,NaN,"United States,United States","S10 OD023579,R01 HL140468","stroke,diastole,heart failure,renal dysfunction",NaN,"journal article,research support\, n.i.h.\, ex...",NaN,"university,university,university,university,un...","adrenergic beta-antagonists,autoantigens,bioma...",...,NaN,NaN,American Journal of Physiology - Heart and Cir...,American Journal of Physiology - Heart and Cir...,university of miami leonard m. miller school o...,NaN,"adrenergic beta-antagonists,animals,autoantige...","Chemicals and Drugs,Organisms,Analytical\, Dia...",33769915.0,"col4a3-/- alport mouse,urine albumin,carvedilo..."
4,2021-08-06T01:24:53Z,NaN,NaN,PIMS Code:OXX01915,mortality,"nagaland centre,nagaland centre,nagaland centre",journal article,NaN,"center,center,center",chickens,...,NaN,NaN,Tropical Animal Health and Production,Tropical Animal Health and Production,"nagaland centre,nagaland centre,nagaland centre","india,india,india,india","animals,chickens,female,india,male,ovum,tropic...","Organisms,Anatomy,Phenomena and Processes,Heal...",33770287.0,"north eastern hill,chicken,yolk,srinidhi,germp..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2021-08-06T01:55:23Z,NaN,NaN,NaN,"neurological disorder,diabetes mellitus,fractu...",NaN,"journal article,multicenter study,observationa...",NaN,"university,university,university,hospital,hosp...","greece,malnutrition,mass screening,nutritional...",...,"""p. and a. kyriakou"" children's hospital,""p. a...",NaN,Nutrients,Nutrients,"harokopio university,harokopio university,haro...","greece,greece,greece,greece,greece,greece,gree...","child,child\, preschool,cross-sectional studie...","Analytical\, Diagnostic and Therapeutic Techni...",33924630.0,"pyms,clinical setting study,dietitian,whogc,su..."
4996,2021-08-06T01:55:23Z,jagiellonian university medical college,NaN,number NRSA 406754,"hyporesponsiveness,rheumatoid arthritis,allerg...",NaN,"journal article,randomized controlled trial",NaN,"university,university,university,university,un...","adaptation\, physiological,athletic performanc

## combining Title and preprocessing


In [39]:

# remove special characters 
def remove_special(x):
    # new line at sentence end for training purposes
    x = x.replace('.', '\n')
    return re.sub('[^A-Za-z \n]+', ' ', x).lower()
    
stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)
def remove_stopwords(x):
    return ' '.join([word for word in x.split() if word not in stopwords_dict])

abstract = pat_data['Abstract'][~pat_data['Abstract'].isna()].apply(remove_stopwords).apply(remove_special).values
title = pat_data['Title'][~pat_data['Title'].isna()].apply(remove_stopwords).apply(remove_special).values
corpus = np.concatenate([title,abstract], axis=0)
data = '\n'.join(corpus)


In [46]:

#### patbase cleaning
def text_cleaning(text):
    
    cleaned_txt = re.sub('<[^<]+>', ' ', str(text))
    cleaned_txt = re.sub( r'\[.*?\]', ' ',cleaned_txt)  #remove brackets
    cleaned_txt = re.sub('\d+', ' ', cleaned_txt)
    cleaned_txt = re.sub('\(\w+\)', ' ', cleaned_txt) 
  
#     cleaned_txt=cleaned_txt.translate(str.maketrans(' ', ' ',string.punctuation)) 
    return cleaned_txt
   
def stopword_removal(text):
    cleaned_txt = ' '.join([token for token in word_tokenize(text) if token not in stop_words])
    return cleaned_txt
def lemmatize_text(text):
    cleaned_txt=' '.join([lemmatizer.lemmatize(token) for token in word_tokenize(text)])
    return cleaned_txt

def noun_adj_fetcher(cleaned_txt):
    Noun_adj=[w for w , pos in pos_tag(str(cleaned_txt).split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )]
    return Noun_adj

# python -m spacy download en_core_web_sm
def noun_adj_pairs(text):
    doc = nlp(str(text))
    noun_adj_pairs = []
    for i,token in enumerate(doc):
        if token.pos_ not in ('NOUN','PROPN'):
            continue
        for j in range(i+1,len(doc)):
            if doc[j].pos_ == 'ADJ':
                noun_adj_pairs.append((token,doc[j]))
                break
    return noun_adj_pairs

def preprocessed_text(df):
    
#     col=['nltk_tokens','nouns','lemmatize_text']
    preprocess_txt=pd.DataFrame()
    abs_clean_text=df.apply(text_cleaning)
    preprocess_txt['lemmatize_text']=abs_clean_text.apply(stopword_removal)         
#     preprocess_txt['lemmatize_text']=stopword_removed_text.apply(lemmatize_text)
  
#     for ind,text in preprocess_txt['lemmatize_text'].iteritems():       
#         tokens=word_tokenize(text)
#         pos_tokens=nltk.pos_tag(tokens)
# #         preprocess_txt.at[ind,'nouns']=[pos_token for pos_token,pos in pos_tokens if pos.startswith('N')]
# #         preprocess_txt.at[ind,'nltk_tokens']=tokens     
    return preprocess_txt


In [48]:
abs_text=pat_data['Abstract']

# abs_text=pat_data['Title']
# abs_text[1]

preprocessed_df1=preprocessed_text(abs_text)
preprocessed_df1
# preprocessed_df1['Noun_adj']=abs_text.apply(noun_adj_pairs)
# corpus='\n'.join([i for i in preprocessed_df1['lemmatize_text']])
# print((corpus[:100]))
# corpus=' '.join(preprocessed_df1['lemmatize_text'])
# corpus[:100]

,lemmatize_text
0,contamination meat antimicrobial-resistant bac...
1,g-protein-coupled receptor ( gpr ) considered ...
2,structures melatonin ferulic acid merged terti...
3,currently food drug administration-approved tr...
4,productivity traditional backyard poultry deve...
...,...
4995,nutritional risk screening yet established man...
4996,purpose : growing number studies indicate impo...
4997,non-alcoholic-fatty liver disease spreading wo...
4998,propose test model food policy acceptability ....


In [50]:
sample_pat_data=preprocessed_df1['lemmatize_text'][0]
sample_pat_data=text_cleaning(sample_pat_data).lower()
print(sample_pat_data)


class Sentence(NamedTuple):
    """
    a data model for a sentence implemented using NamedTuple
    """
    words: List[str]
    pos: List[str]
    lemmas: List[str]
    length: int


class Document(NamedTuple):
    """
    a data model for document implemented using NamedTuple
    """
    sentences: List[Sentence]

def process(text, max_length=None):
    """
        given a text, call spacy for sentence splitting, lemmatisation, and POS tagging
        :param text:
        :param max_length:
        :return: a document with processed sentences
    """
    nlp = spacy.load('en_core_web_sm')
    
    doc = nlp(text)
    
    sentences = []
    for sent in doc.sents:
        sentences.append({
                "words": [token.text for token in sent],
                "lemmas": [token.lemma_ for token in sent],
                "pos": [token.pos_ for token in sent],
        })

    return Document(sentences=[Sentence(words=s['words'], pos=s['pos'], lemmas=s['lemmas'], length=len(s['words']))
                                   for s in sentences])

contamination meat antimicrobial-resistant bacteria represents major public health threat worldwide . study\ , determined antimicrobial resistance profiles resistance trends staphylococcus aureus isolated major food animal carcasses ( cattle\ , pig\ , chicken carcass isolates ) korea . approximately % \ , % \ , % cattle\ , pig\ , chicken carcass isolates\ , respectively\ , resistant least one antimicrobial agent . resistance penicillin ( . % ) highest\ , followed resistance tetracycline ( . % ) erythromycin ( . % ) . % pig chicken isolates resistant ciprofloxacin . observed linezolid resistance pig isolates ( . % ) . however\ , s. aureus isolates sensitive rifampin vancomycin . noted increasing fluctuating trend kanamycin penicillin resistance cattle isolates . similarly\ , chloramphenicol\ , ciprofloxacin\ , tetracycline\ , trimethoprim resistance rates increased fluctuated time pig isolates . methicillin-resistant s. aureus accounted % \ , % \ , % cattle\ , pig\ , chicken isolates\ ,

In [51]:
def build_word_graph(window=None, pos=None):
    graph = nx.Graph()
    
    # use lemmas to build a word graph
    text = [(word, sentence.pos[i] in pos) for sentence in sentences for i, word in enumerate(sentence.lemmas)]

    # add nodes
    graph.add_nodes_from([word for word, valid in text if valid])

    # add edges
    for i, (node1, is_valid1) in enumerate(text):
        # skip if a word is neither a noun nor an adjective
        if not is_valid1:
            continue

        # add an edge with node2 within a window size
        for j in range(i + 1, min(i + window, len(text))):
            node2, is_valid2 = text[j]
            if is_valid2 and node1 != node2:
                graph.add_edge(node1, node2)
    return graph

In [52]:
doc=process(sample_pat_data,max_length=10**5)
sentences=doc.sentences
VALID_POSTAGS = {'ADJ','NOUN'}
graph=build_word_graph(window=3,pos=VALID_POSTAGS)
graph

In [53]:
def select_longest_keyword_sequences(keywords):
    """
    Select longest sequences of keywords
    :param keywords: a list of keywords used for building a word graph
    :return:
    """
    for i, sentence in enumerate(sentences):
        # a list of valid token offsets
        sequence = []
        candi=[]
        for j, token in enumerate(sentences[i].lemmas):
            # add an offset of a valid token into longest_sequence
            if token in keywords:
                sequence.append(j)
                if j < (sentence.length - 1):
                    continue
            # when a token is not valid check if there is a sequence
            if sequence:
                start = sequence[0]
                end = sequence[-1] + 1
             
                cand=add_candidate(words=sentence.words[start:end],
                                       lemmas=sentence.lemmas[start:end],
                                       pos=sentence.pos[start:end],
                                       sentence_id=i)
                
                candi.append(cand)
            sequence=[]
        return candi
# # reset the list
#     sequence = []   

In [54]:
class Candidate(object):
    """
    data model for keyword/phrase candidates
    """
    def __init__(self):
        """
        variable initialization
        """
        self.lexical_form = []
        self.surface_forms = []
        self.sentence_ids = []
        self.pos_patterns = []
    
    def add_candidate(words, lemmas, pos, sentence_id):
        """
            populate candidates for a document
            :param words:
            :param lemmas:
            :param pos:
            :param sentence_id:
            :return:
        """
        candidates = defaultdict(Candidate)
        key = ' '.join(words).lower()

        candidates[key].lexical_form = lemmas
        candidates[key].surface_forms.append(words)
        candidates[key].pos_patterns.append(pos)
        candidates[key].sentence_ids.append(sentence_id)
        return candidates

In [55]:
def rank_candidates(normalized=False):
        """
        Rank the longest sequences
        :param normalized:
        :return:
        """
        weights = {}
        # calculate PageRank score for each node using default values
        w = nx.pagerank_numpy(graph, alpha=0.5, weight=None)
#         print(w)
        wt=0
        keywords = ','.join(sorted(w, key=w.get, reverse=True))
#         print('keywords',keywords)
        
        cand=select_longest_keyword_sequences(keywords)
#         print('cand:',cand)
        tokens= [j for i in cand for j in i.keys()]
#         print('tokens',tokens)
        for i in tokens:
            for j in i.split():
                if j in w.keys():
                    wt+=w[j]
                    weights[i]=wt
                    if normalized:
                        weights[i] /= len(tokens)            
        return weights,cand,keywords

In [57]:
weights,cand,keywords=rank_candidates(normalized=True)
weights

<ipython-input-55-1ed89d36944c>:9: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  w = nx.pagerank_numpy(graph, alpha=0.5, weight=None)


{'contamination meat antimicrobial - resistant bacteria': 0.0673646975523209,
 'major public health threat': 0.11124821143325522}

In [58]:
col=['Abstract','Keyword_extracted','Keywords_ranked']
keyword_tagging_df=pd.DataFrame(columns=col)
keyword_tagging_df['Abstract']=pat_data['Abstract']
keyword_tagging_df['Keyword_extracted']=pat_data['keyword_copy_txt']


    
for index,text in tqdm(enumerate(preprocessed_df1['lemmatize_text'][:5])):        
    doc=process(text,max_length=10**5)
    sentences=doc.sentences
    VALID_POSTAGS = {'NOUN'}
    graph=build_word_graph(window=3,pos=VALID_POSTAGS)
    weights,cand,keywords=rank_candidates(normalized=False)
    keyword_tagging_df.loc[index].Keywords_ranked=keywords
    
#     cons_keys=consolidate_duplicate_candidates(threshold=0.9)
#     print('consolidated keys',cons_keys)

0it [00:00, ?it/s]<ipython-input-55-1ed89d36944c>:9: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  w = nx.pagerank_numpy(graph, alpha=0.5, weight=None)
5it [00:03,  1.36it/s]


In [59]:
# keyword_tagging_df.to_csv('V:/ML_projects/Merckgroup/Keyword_tagging/preprocess_output/.csv')
keyword_tagging_df

,Abstract,Keyword_extracted,Keywords_ranked
0,contamination of meat with antimicrobial-resis...,"s. aureus,antimicrobial resistance,carcasses,f...","resistance,trend,isolate,carcass,health,pig,%,..."
1,g-protein-coupled receptor 40 (gpr40) is consi...,NaN,"gpr,acid,secretion,β,type,gsis,target,agonist,..."
2,the structures of melatonin and ferulic acid w...,NaN,"hdac,melatonin,ability,inhibitor,dysfunction,s..."
3,there are currently no food and drug administr...,"hfpef,carvedilol,diastolic dysfunction,exercis...","carvedilol,alport,treatment,function,col,model..."
4,productivity of traditional backyard poultry i...,"backyard poultry production,north eastern hill...","parent,vanaraja,week,mortality,egg,backyard,ca..."
...,...,...,...
4995,nutritional risk screening (nrs) is not yet es...,"pyms,stamp,who,malnutrition,nutritional risk,p...",NaN
4996,purpose: a growing number of studies indicate ...,"il-6,inflammation,resistin,skeletal muscle dam...",NaN
4997,non-alcoholic-fatty liver disease (nafld) is s...,"nafld,gut microbiome,insulin resistance,metabo...",NaN
4998,we propose and test a model of food policy acc...,"acceptability,food policy,survey",NaN


In [60]:
def consolidate_duplicate_candidates(threshold=None):
    """
    Filter the candidates
    :return:
    """
    weights,candidates,keyword=rank_candidates(normalized=False)
    
    candidates_list=[j for i in candidates for j in i.keys()]
    
    top_keys = sorted(weights, key=weights.get, reverse=True)
#     print('''\n \ntop keys\n''',top_keys)
    w = nx.pagerank_numpy(graph, alpha=0.5, weight=None)
#     print(len([i.keys() for i in candidates]))
    
    keywords = sorted(w, key=w.get, reverse=True)
    candidates=select_longest_keyword_sequences(keywords)

    keys_to_consolidate = []
    
    for (k1, k2) in combinations(top_keys, 2):

        if k1 == k2:
            continue
        # calculate distance between key phrases
#         distance = model.wv.wmdistance(k1, k2)
# #         print('distance',distance)
#         print(k1,'dist',k2,'-->',distance)
#         # remove a lower ranked candidate
#         if distance > threshold:
#             keys_to_consolidate.append((k1,k2))
# #     print('keys to consolidate',keys_to_consolidate)

#     return(keys_to_consolidate)


In [61]:
consolidate_duplicate_candidates(threshold=0.02)

<ipython-input-55-1ed89d36944c>:9: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  w = nx.pagerank_numpy(graph, alpha=0.5, weight=None)
<ipython-input-60-7b6cdd2c7a6b>:12: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  w = nx.pagerank_numpy(graph, alpha=0.5, weight=None)


### MMR

In [62]:
top_n=15
diversity=0.3
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")


In [63]:
def mmr(doc_embedding: np.ndarray,word_embeddings: np.ndarray,words,top_n,diversity):
    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)
    
    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
    
    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
#         mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        
        if mmr.size>0:
            mmr_idx = candidates_idx[np.argmax(mmr)]
            
            # Update keywords & candidates
            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)
            
    return [(words[idx], round((word_doc_similarity.reshape(1, -1)[0][idx]), 4)) for idx in keywords_idx]

In [13]:
def cv_word_doc_embedding(doc):
    cv_countvec = CountVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit([doc])
    cv_words = cv_countvec.get_feature_names()
    cv_doc_embeddings = embedding_model.encode([doc])
    cv_word_embeddings =embedding_model.encode(cv_words)
    return cv_word_embeddings,cv_doc_embeddings,cv_words
    
def tfidf_word_doc_embedding(doc):
    tfidf_countvec = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=0.8).fit([doc])
    tfidf_words = tfidf_countvec.get_feature_names()    

    tfidf_doc_embeddings = embedding_model.encode([doc])
    tfidf_word_embeddings = embedding_model.encode(tfidf_words)
    return tfidf_word_embeddings,tfidf_doc_embeddings,tfidf_words

In [14]:
def text_cleaning(text):
    cleaned_txt = re.sub('<[^<]+>', ' ', str(text))
    cleaned_txt = re.sub( r'\[.*?\]', ' ',cleaned_txt)  #remove brackets
#     cleaned_txt = re.sub('\d+', ' ', cleaned_txt)
#     cleaned_txt = re.sub('\(\w+\)', ' ', cleaned_txt)
    cleaned_txt=text.translate(str.maketrans(' ', ' ',string.punctuation)) 
    return cleaned_txt
   
def stopword_removal(text):
    cleaned_txt = ' '.join([token for token in word_tokenize(text) if token not in stop_words])
    return cleaned_txt
def lemmatize_text(text):
    cleaned_txt=' '.join([lemmatizer.lemmatize(token) for token in word_tokenize(text)])

    return cleaned_txt

def preprocessed_text(df):
    
    col=['nltk_tokens','nouns_verbs','lemmatize_text']
    preprocess_txt=pd.DataFrame(columns=col)
    abs_clean_text=df.apply(text_cleaning)
    
    stopword_removed_text=abs_clean_text.apply(stopword_removal)         
    preprocess_txt['lemmatize_text']=stopword_removed_text.apply(lemmatize_text)
  
    for ind,text in preprocess_txt['lemmatize_text'].iteritems():       
        tokens=word_tokenize(text)
        pos_tokens=nltk.pos_tag(tokens)
        preprocess_txt.at[ind,'nouns_verbs']=[pos_token for pos_token,pos in pos_tokens if pos.startswith('N' or 'V')]
        preprocess_txt.at[ind,'nltk_tokens']=tokens     
    return preprocess_txt

In [15]:
doc=preprocessed_df1['lemmatize_text'][:100]

tfidf_keywords=[]
tfidf_phrases=[]
for i in tqdm(doc):
    tfidf_word_embed,tfidf_doc_embed,tfidf_words=tfidf_word_doc_embedding(i)
    tfidf_keywords_doc=mmr(tfidf_doc_embed,tfidf_word_embed,tfidf_words,5,0.2)
    tfidf_keywords_doc=','.join([key[0] for key in tfidf_keywords_doc])
    
    tfidf_keywords.append(tfidf_keywords_doc)
#     tfidf_phrases.append(tfidf_phrases_doc)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.36it/s]


In [16]:
tfidf_keywords[:15]

['antimicrobials,pig,resistance,isolates,bacteria',
 'insulin,gpr,receptor,glycemic,diabetes',
 'inhibitors,melatonin,hdac,antioxidant,nm',
 'dysfunction,cardiac,adrenergic,drug,renal',
 'poultry,chicken,srinidhi,backyard,hen',
 'staining,brains,samples,cadaver,clearing',
 'angiotensin,immunosuppression,fibroblasts,antitumor,macrophages',
 'crystal,prediction,framework,organic,molecule',
 'hydrogels,gels,biomacromolecules,tissues,gel',
 'food,simulation,powders,heat,compositional',
 'pore,adsorption,porosimetry,nanoporous,physisorption',
 'stigma,dietary,severity,levels,health',
 'mutations,mpl,myelofibrosis,exon,mutation',
 'sport,risks,statistical,association,reviewed',
 'neuropathy,elbow,myositis,nerves,pain']

In [19]:
cv_keywords=[]
cv_phrases=[]
for i in tqdm(doc):
    cv_word_embed,cv_doc_embed,cv_words=cv_word_doc_embedding(i)
    cv_keywords_doc=mmr(cv_doc_embed,cv_word_embed,cv_words,10,0.2)
    cv_keywords_doc=','.join([key[0] for key in cv_keywords_doc])
    
    cv_keywords.append(cv_keywords_doc)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 25.19it/s]


In [20]:
# tfidf_keywords
# preprocessed_df1['tfidf_mmr']=tfidf_keywords
cv_keywords[:15]

['antimicrobials,pig,resistance,isolates,bacteria,aureus,carcass,risk,staphylococcus,contamination',
 'insulin,gpr,receptor,glycemic,diabetes,peptide,glp,drug,gsis,glucose',
 'inhibitors,melatonin,hdac,antioxidant,nm,aβ,deacetylase,ic,acid,neurotoxicity',
 'dysfunction,cardiac,adrenergic,drug,renal,hfpef,cardiorenal,levels,activity,mice',
 'poultry,chicken,srinidhi,backyard,hen,reproductive,growth,climatic,vanaraja,birds',
 'staining,brains,samples,cadaver,clearing,antibody,optimized,microscopy,evaluating,quality',
 'angiotensin,immunosuppression,fibroblasts,antitumor,macrophages,receptor,therapeutic,tumor,immunity,chemokine',
 'crystal,prediction,framework,organic,molecule,research,pharmaceutical,methodologies,agrochemical,theoretical',
 'hydrogels,gels,biomacromolecules,tissues,gel,toughening,tendon,bonds,biomedical,ligament',
 'food,simulation,powders,heat,compositional,agglomeration,industry,mass,flowability,model',
 'pore,adsorption,porosimetry,nanoporous,physisorption,porous,prot

### Multiple doc

In [37]:
def phrase_generator(docs,words,countvec):
    
    top_n:int=10
    diversity:0.5
    df = countvec.transform(docs)
    
    keywords=[]    
    phrases=[]
    
    
    for index, doc in enumerate(docs):
        doc_words = [words[i] for i in df[index].nonzero()[1]]
        if doc_words:
            doc_word_embeddings = np.array([tfidf_word_embeddings[i] for i in df[index].nonzero()[1]])
            distances = cosine_similarity([tfidf_doc_embeddings[index]], doc_word_embeddings)[0]
            doc_keywords = [(doc_words[i], round(float(distances[i]), 4)) for i in distances.argsort()[-top_n:]]
            doc_keywords=','.join([key[0] for key in doc_keywords])
            doc_phrases=''.join([key[0] for key in doc_keywords])
            keywords.append(doc_keywords)
            phrases.append(doc_phrases)                                 
    return keywords,phrases

In [106]:
docs1=preprocessed_df1['lemmatize_text']
tfidf_countvec = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit(docs1)
tfidf_words = tfidf_countvec.get_feature_names()

tfidf_doc_embeddings = embedding_model.encode(docs1)
tfidf_word_embeddings = embedding_model.encode(tfidf_words)

tfidf_keywords_doc,tfidf_phrases_doc=phrase_generator(docs1,tfidf_words,tfidf_countvec)



In [7]:
path='../word_embedding/bio_embedding_intrinsic.bin'
Bioword_intrinsic_model=KeyedVectors.load_word2vec_format(path,binary=True)

In [324]:
# import kmapper as km
# # Project all of the "sentences" down into a 2D representation
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.manifold import Isomap
# from sklearn.preprocessing import MinMaxScaler
# from sklearn import cluster

# def keyphrase_mapper(phrases):
#     mapper = km.KeplerMapper(verbose=2)

#     projected_X = mapper.fit_transform(np.array(phrases),
#     projection=[TfidfVectorizer(analyzer="char",
#                             ngram_range=(1,6),
#                             max_df=0.85,
#                             min_df=0.05),
#             TruncatedSVD(n_components=100,
#                          random_state=1729),
#             Isomap(n_components=2,
#                    n_jobs=-1)],
#     scaler=[None, None, MinMaxScaler()])

#     print("SHAPE",projected_X.shape)

#     return projected_X

# # Cluster the projected data
# def cluster_visualization(docs,projected_X,path):
#     mapper = km.KeplerMapper(verbose=2)
#     graph = mapper.map(projected_X, clusterer=cluster.DBSCAN(eps=0.5, min_samples=3))
#     # Get counts of phrases from data (to be used for the label and for cluster statistics)
#     vec = TfidfVectorizer(analyzer="word",
#                   strip_accents="unicode",
#                   stop_words="english",
#                   ngram_range=(1,3),
#                   max_df=0.9,
#                   min_df=0.02)

#     interpretable_inverse_X = vec.fit_transform(docs).toarray()
#     interpretable_inverse_X_names = vec.get_feature_names()

#     print("SHAPE", interpretable_inverse_X.shape)
#     print("FEATURE NAMES SAMPLE",
#     interpretable_inverse_X_names[:400])
#     html = mapper.visualize(graph,X=interpretable_inverse_X,
#                         X_names=interpretable_inverse_X_names,
#                         path_html=path,
#                         lens=projected_X,
#                         lens_names=["ISOMAP1", "ISOMAP2"],
#                         title="Visualizing Text - Abstract",
#                         )

# projected_tfidf_X=keyphrase_mapper(tfidf_phrases_doc)

# cluster_visualization(docs1,projected_tfidf_X,'Abstract_tfidf_mmr_final.html')

In [325]:
def consolidate_duplicate_candidates(threshold=None):
    """
    Filter the candidates
    :return:
    """

    weights,candidates,keyword=rank_candidates(normalized=False)
    
    candidates_list=[j for i in candidates for j in i.keys()]
    
    top_keys = sorted(weights, key=weights.get, reverse=True)
    print('''\n \ntop keys\n''',top_keys)
    w = nx.pagerank_numpy(graph, alpha=0.5, weight=None)
    print(len([i.keys() for i in candidates]))
    
    keywords = sorted(w, key=w.get, reverse=True)
    candidates=select_longest_keyword_sequences(keywords)

    keys_to_consolidate = []
    reasons_to_consolidate =[]
    
    for (k1, k2) in combinations(top_keys, 2):

        if k1 == k2:
            continue
        # calculate distance between key phrases
#         distance =Bioword_intrinsic_model.wmdistance(k1,k2)
        distance =Bioword_intrinsic_model.wmdistance(k1,k2)
        print('distance',distance)
        print(k1,'dist',k2,'-->',distance)
        # remove a lower ranked candidate
        if distance > threshold:
            keys_to_consolidate.append(k2)
            reasons_to_consolidate.append((k2, k1))
    # delete candidates selected from above
#     print(candidates_list)
#     keys_to_delete=list(set(keys_to_delete))
    print('keys to consolidate',keys_to_consolidate)
#     res = list(filter(lambda i: i not in keys_to_delete, candidates_list))
#     print('res',res)

In [49]:
# consolidate_duplicate_candidates(threshold=0.01)

condition dist backyard chicken --> 0.027770979799234596
condition dist introduction --> 0.00914646026217833
condition dist backyard poultry --> 0.029565351907635918
backyard chicken dist introduction --> 0.02635782929161031
backyard chicken dist backyard poultry --> 0.017078582547482336
introduction dist backyard poultry --> 0.023584520742763463


[('condition', 'backyard chicken'),
 ('condition', 'backyard poultry'),
 ('backyard chicken', 'introduction'),
 ('backyard chicken', 'backyard poultry'),
 ('introduction', 'backyard poultry')]

## Transformer model

In [49]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

from transformers import DistilBertTokenizer, DistilBertModel

In [50]:
%%time
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=64)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=64, activation_function=nn.Tanh())
sent_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Wall time: 19.9 s


In [43]:
# %%time
# word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
# sent_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Wall time: 17.5 s


In [8]:
%%time
word_embedding_model = models.Transformer('distilbert-base-uncased', max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
sent_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Wall time: 13.8 s


## Single Doc MMR


In [65]:
def tfidf_word_doc_embedding(doc):
    tfidf_countvec = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=0.8).fit([doc])
    tfidf_words = tfidf_countvec.get_feature_names()    

    tfidf_doc_embeddings = sent_model.encode([doc])
    tfidf_word_embeddings = sent_model.encode(tfidf_words)
    return tfidf_word_embeddings,tfidf_doc_embeddings,tfidf_words

In [68]:
doc=preprocessed_df1[:10]
tfidf_BERT_keywords=[]
for i in tqdm(doc):
    tfidf_word_embed,tfidf_doc_embed,tfidf_words=tfidf_word_doc_embedding(i)
    tfidf_keywords_doc=mmr_consolidate(tfidf_doc_embed,tfidf_word_embed,tfidf_words,5,0.2)
    tfidf_keywords_doc=','.join([key[0] for key in tfidf_keywords_doc])    
    tfidf_BERT_keywords.append(tfidf_keywords_doc)
    


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.09it/s]


In [69]:
tfidf_BERT_keywords


['throughput,pooled,crispr,cas9,customizing',
 'prokaryotes,plasmids,epidermidis,deletion,antisense',
 'throughput,aligner,visualization,multiplexing,nxuning',
 'helicase,repressor,transcriptional,nuclease,infect',
 'cytometry,plasmid,qpcr,interspaced,palindromic',
 'biotechnological,prokaryotic,translational,controllable,palindromic',
 'elucidate,epidemiology,epidemiological,antagonize,phage',
 'refseq,throughput,oligonucleotides,nucleotide,sgrnas',
 'photosensor,epigenome,acriia4,streptococcus,spatiotemporally',
 'heuristic,predefined,mismatches,biotechnologies,benchmark']

## Multiple Documents

In [63]:
def phrase_generator(docs,words,countvec):
    
    top_n:int=15
    diversity:0.5
    df = countvec.transform(docs)
    
    keywords=[]    
    
    
    for index, doc in enumerate(tqdm(docs)):
        doc_words = [words[i] for i in df[index].nonzero()[1]]
        if doc_words:
            doc_word_embeddings = np.array([tfidf_word_embeddings_bert[i] for i in df[index].nonzero()[1]])
            distances = cosine_similarity([tfidf_doc_embeddings_bert[index]], doc_word_embeddings)[0]
            doc_keywords = [(doc_words[i], round(float(distances[i]), 4)) for i in distances.argsort()[-top_n:]]
            doc_keywords=[key[0] for key in doc_keywords]
            keywords.append(doc_keywords)
#             phrases.append(doc_phrases)                                 
    return keywords

In [18]:
%%time
docs1=preprocessed_df1['lemmatize_text'][:10]
tfidf_countvec = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit(docs1)
tfidf_words = tfidf_countvec.get_feature_names()

tfidf_doc_embeddings_bert = sent_model.encode(docs1)
tfidf_word_embeddings_bert = sent_model.encode(tfidf_words)

Wall time: 8.59 s


In [21]:
tfidf_keywords_doc_bert=phrase_generator(docs1,tfidf_words,tfidf_countvec)
tfidf_keywords_doc_bert

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 415.60it/s]


[['carcass',
  'carcasses',
  'fluctuating',
  'chloramphenicol',
  'kanamycin',
  'vancomycin',
  'rifampin',
  'erythromycin',
  'ciprofloxacin',
  'staphylococcus',
  'antimicrobial',
  'methicillinresistant',
  'antimicrobials',
  'methicillinsusceptible',
  'antimicrobialresistant'],
 ['agonist',
  'secretory',
  'dbdb',
  'chemotype',
  'pancreatic',
  'propanoic',
  'plasmaactive',
  'agonists',
  'βcells',
  'arylsubstituted',
  'glycemic',
  'acidmediated',
  'glucagonlike',
  'glucosestimulated',
  'gproteincoupled'],
 ['chelating',
  'melatonin',
  'selectivity',
  'longterm',
  'ferulic',
  'deacetylase',
  'equimolar',
  'amidebased',
  'antioxidant',
  'neurotoxicity',
  'neurodegenerative',
  'structureactivity',
  'multitargetdirected',
  'immunomodulatory',
  'diseasemodifying'],
 ['synergistically',
  'creatinine',
  'osteopontin',
  'untreated',
  'synergistic',
  'diastole',
  'echocardiography',
  'cardiorenal',
  'carvedilol',
  'systolic',
  'administrationapprov

In [71]:
tfidf_keywords_doc_bert[0]

['linezolid',
 'fluctuating',
 'kanamycin',
 'tetracycline',
 'rifampin',
 'chloramphenicol',
 'vancomycin',
 'erythromycin',
 'methicillinresistant',
 'antimicrobial',
 'staphylococcus',
 'ciprofloxacin',
 'antimicrobials',
 'methicillinsusceptible',
 'antimicrobialresistant']

## Pretrained Transformer RANKING 
'biomed_RoBERTa-base to 2.68 million scientific papers '

In [10]:
from transformers import AutoModel, AutoTokenizer

In [44]:
# model_name = "distilroberta-base"
model_name='allenai/biomed_roberta_base'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at allenai/biomed_roberta_base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [72]:
text_list=preprocessed_df1['lemmatize_text'].dropna().tolist()
cand=tfidf_keywords_doc_bert[0]

In [74]:

candidate_tokens = tokenizer(cand, padding=True, return_tensors="pt")
candidate_embeddings = model(**candidate_tokens)["pooler_output"]
print(candidate_embeddings)
text_tokens = tokenizer([text_list[0]], padding=True, return_tensors="pt")
text_embedding = model(**text_tokens)["pooler_output"]


candidate_embeddings = candidate_embeddings.detach().numpy()
text_embedding = text_embedding.detach().numpy()

tfidf_keywords_doc=mmr(text_embedding,candidate_embeddings,tfidf_keywords_doc_bert[0],10,0.2)
print(tfidf_keywords_doc)

tensor([[-0.0071, -0.2261, -0.3298,  ..., -0.1019, -0.0607, -0.0847],
        [-0.0174, -0.2169, -0.2836,  ..., -0.0713, -0.0784, -0.1130],
        [-0.0118, -0.2185, -0.3016,  ..., -0.1093, -0.0763, -0.0832],
        ...,
        [-0.0183, -0.2111, -0.3005,  ..., -0.0932, -0.0795, -0.0869],
        [-0.0101, -0.2473, -0.3118,  ..., -0.1012, -0.0755, -0.0712],
        [-0.0158, -0.2245, -0.3067,  ..., -0.1015, -0.0803, -0.0722]],
       grad_fn=<TanhBackward>)
[('methicillinresistant', 0.9887), ('staphylococcus', 0.988), ('antimicrobialresistant', 0.9882), ('methicillinsusceptible', 0.9881), ('antimicrobial', 0.988), ('rifampin', 0.9867), ('antimicrobials', 0.9871), ('chloramphenicol', 0.9865), ('ciprofloxacin', 0.9865), ('tetracycline', 0.9863)]


In [79]:
text_list=preprocessed_df1['lemmatize_text'].dropna().tolist()
len(tfidf_keywords_doc_bert)

100

In [80]:
tfidf_keywords_pre_bert=[]

for index, doc in enumerate(tqdm(docs1[:100])):
    candidate_tokens = tokenizer(tfidf_keywords_doc_bert[index], padding=True, return_tensors="pt",truncation=True)
    
    candidate_embeddings = model(**candidate_tokens)["pooler_output"]
    
    candidate_embeddings = candidate_embeddings.detach().numpy()
    
    text_tokens = tokenizer([text_list[index]], padding=True, return_tensors="pt",truncation=True)
    text_embedding = model(**text_tokens)["pooler_output"]
    text_embedding = text_embedding.detach().numpy()
    
    tfidf_keywords_doc_mmr=mmr(text_embedding,candidate_embeddings,tfidf_keywords_doc_bert[index],15,0.2)
    
    tfidf_keywords_pre_bert.append(tfidf_keywords_doc_mmr)




  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  1%|▊                                                                                 | 1/100 [00:01<02:51,  1.73s/it]

  2%|█▋                                                                                | 2/100 [00:02<02:28,  1.52s/it]

  3%|██▍                                                                               | 3/100 [00:03<02:13,  1.37s/it]

  4%|███▎                                                                              | 4/100 [00:05<02:14,  1.40s/it]

  5%|████                                                                              | 5/100 [00:06<02:11,  1.38s/it]

  6%|████▉                                                                             | 6/100 [00:07<02:03,  1.31s/it]

  7%|█████▋                                                                            | 7/100 [00:09<02:05,  1.35s/it]

  8%|██████▌                  

In [28]:
tfidf_keywords_pre_bert[0]

[('antimicrobial', 0.9893),
 ('staphylococcus', 0.9888),
 ('methicillin', 0.9887),
 ('antimicrobials', 0.9887),
 ('isolates', 0.9878),
 ('penicillin', 0.988),
 ('linezolid', 0.9873),
 ('ciprofloxacin', 0.9874),
 ('tetracycline', 0.9874),
 ('chloramphenicol', 0.9873),
 ('kanamycin', 0.9871),
 ('trimethoprim', 0.9871),
 ('erythromycin', 0.9869),
 ('vancomycin', 0.9868),
 ('fluctuating', 0.9856)]

In [4]:
def maximal_marginal_relevance(sentence_vector, phrases, embedding_matrix, lambda_constant=0.5, threshold_terms=10):
    """
    Return ranked phrases using MMR. Cosine similarity is used as similarity measure.
    :param sentence_vector: Query vector
    :param phrases: list of candidate phrases
    :param embedding_matrix: matrix having index as phrases and values as vector
    :param lambda_constant: 0.5 to balance diversity and accuracy. if lambda_constant is high, then higher accuracy. If lambda_constant is low then high diversity.
    :param threshold_terms: number of terms to include in result set
    :return: Ranked phrases with score
    """
    # todo: Use cosine similarity matrix for lookup among phrases instead of making call everytime.
    s = []
    r = sorted(phrases, key=lambda x: x[1], reverse=True)
    r = [i[0] for i in r]
    while len(r) > 0:
        score = 0
        phrase_to_add = ''
        for i in r:
            first_part = cosine_similarity([sentence_vector], [embedding_matrix.loc[i]])[0][0]
            second_part = 0
            for j in s:
                cos_sim = cosine_similarity([embedding_matrix.loc[i]], [embedding_matrix.loc[j[0]]])[0][0]
                if cos_sim > second_part:
                    second_part = cos_sim
            equation_score = lambda_constant*(first_part)-(1-lambda_constant) * second_part
            if equation_score > score:
                score = equation_score
                phrase_to_add = i
        if phrase_to_add == '':
            phrase_to_add = i
        r.remove(phrase_to_add)
        s.append((phrase_to_add, score))
    return (s, s[:threshold_terms])[threshold_terms > len(s)]

In [5]:
def phrase_generator(docs,words):
    
    top_n:int=15
    diversity:0.5
    
    keywords=[]    
    
    
    for index, doc in enumerate(tqdm(docs)):
        doc_words = [words[i] for i in df[index].nonzero()[1]]
        if doc_words:
            doc_word_embeddings = np.array([tfidf_word_embeddings_bert[i] for i in df[index].nonzero()[1]])
            distances = cosine_similarity([tfidf_doc_embeddings_bert[index]], doc_word_embeddings)[0]
            doc_keywords = [(doc_words[i], round(float(distances[i]), 4)) for i in distances.argsort()[-top_n:]]
            doc_keywords=[key[0] for key in doc_keywords]
            keywords.append(doc_keywords)
#             phrases.append(doc_phrases)                                 
    return keywords